In [1]:
import pandas as pd
import numpy as np
from irec.environment.loader import FullData

In [2]:
from irec.recommendation.agents import SimpleAgent
from irec.recommendation.agents.action_selection_policies import ASPEGreedy, ASPGreedy
import sys
sys.path.append("..")
from action_selection_policies import ASPMismatchEgreedy, ASPMismatchGreedy

In [3]:
from irec.recommendation.agents.value_functions import LinearEGreedy, PTS

In [4]:
from value_functions import ContextualLinUCB, ContextualEgreedy, ContextualPTS, LinUCB

In [5]:
from irec.offline_experiments.evaluation_policies import FixedInteraction
from irec.offline_experiments.metric_evaluators import UserCumulativeInteraction, StageIterations
from irec.offline_experiments.metrics import Hits

### Load Dataset

In [6]:
# Dataset
dataset = {
    'path': "../datasets/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "global", 'train_size': 0.8, 'test_consumes': 5}
validation = {'validation_size': 0.2}
# Loader
loader = FullData(dataset, splitting, validation)
train_dataset, test_dataset, x_validation, y_validation = loader.process()


Applying splitting strategy: global

Test shape: (16892, 4)
Train shape: (80393, 4)

Generating x_validation and y_validation: 
Test shape: (15729, 4)
Train shape: (61345, 4)


### Creating the agents

In [7]:
params = {
    "LinearEGreedy": {"item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "ContextualEgreedy_Te0": {"item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "ContextualEgreedy_Tge0": {"item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "PTS": {"num_lat": 20, "num_particles": 5, "var": 0.5, "var_u": 1.0, "var_v": 1.0},
    "ContextualPTS_Te0": {"num_lat": 20, "num_particles": 5, "var": 0.5, "var_u": 1.0, "var_v": 1.0},
    "ContextualPTS_Tge0": {"num_lat": 20, "num_particles": 5, "var": 0.5, "var_u": 1.0, "var_v": 1.0},
    "LinUCB": {"alpha": 0.5, "num_lat": 10},
    "ContextualLinUCB_Te0": {"alpha": 0.5, "num_lat": 10},
    "ContextualLinUCB_Tge0": {"alpha": 0.5, "num_lat": 10}
}

In [8]:
agents = []

# LinearEGreedy
vf = LinearEGreedy(**params["LinearEGreedy"])
asp = ASPEGreedy(epsilon=0.1)
agent = SimpleAgent(vf, asp, name="LinearEGreedy")
agents.append(agent)

vf = ContextualEgreedy(**params["ContextualEgreedy_Te0"])
asp = ASPMismatchEgreedy(epsilon=0.1, k=0)
agent = SimpleAgent(vf, asp, name="ContextualEgreedy_Te0")
agents.append(agent)

vf = ContextualEgreedy(**params["ContextualEgreedy_Tge0"])
asp = ASPMismatchEgreedy(epsilon=0.1, k=5)
agent = SimpleAgent(vf, asp, name="ContextualEgreedy_Tge0")
agents.append(agent)

# PTS
vf = PTS(**params["PTS"])
asp = ASPGreedy()
agent = SimpleAgent(vf, asp, name="PTS")
agents.append(agent)

vf = ContextualPTS(**params["ContextualPTS_Te0"])
asp = ASPMismatchGreedy(k=0)
agent = SimpleAgent(vf, asp, name="ContextualPTS_Te0")
agents.append(agent)

vf = ContextualPTS(**params["ContextualPTS_Tge0"])
asp = ASPMismatchGreedy(k=20)
agent = SimpleAgent(vf, asp, name="ContextualPTS_Tge0")
agents.append(agent)

# LinUCB
vf = LinUCB(**params["LinUCB"])
asp = ASPGreedy()
agent = SimpleAgent(vf, asp, name="LinUCB")
agents.append(agent)

vf = ContextualLinUCB(**params["ContextualLinUCB_Te0"])
asp = ASPMismatchGreedy(k=0)
agent = SimpleAgent(vf, asp, name="ContextualLinUCB_Te0")
agents.append(agent)

vf = ContextualLinUCB(**params["ContextualLinUCB_Tge0"])
asp = ASPMismatchGreedy(k=20)
agent = SimpleAgent(vf, asp, name="ContextualLinUCB_Tge0")
agents.append(agent)


In [9]:
[agent.name for agent in agents]

['LinearEGreedy',
 'ContextualEgreedy_Te0',
 'ContextualEgreedy_Tge0',
 'PTS',
 'ContextualPTS_Te0',
 'ContextualPTS_Tge0',
 'LinUCB',
 'ContextualLinUCB_Te0',
 'ContextualLinUCB_Tge0']

### Getting the recommendations

In [10]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [11]:
interactions = {}
for agent in agents:
    print(agent.name)
    agent_interactions, action_info = eval_policy.evaluate(agent, train_dataset, test_dataset)
    interactions[agent.name] = agent_interactions

LinearEGreedy
Starting LinearEGreedy Training


rmse=0.800: 100%|██████████| 20/20 [00:21<00:00,  1.07s/it]


Ended LinearEGreedy Training


LinearEGreedy: 100%|██████████| 18900/18900 [00:14<00:00, 1326.83it/s]


ContextualEgreedy_Te0
Starting ContextualEgreedy_Te0 Training


...

LinUCB: 100%|██████████| 18900/18900 [00:16<00:00, 1151.09it/s]


ContextualLinUCB_Te0
Starting ContextualLinUCB_Te0 Training
0.0 1.0
Ended ContextualLinUCB_Te0 Training


ContextualLinUCB_Te0: 100%|██████████| 18900/18900 [00:14<00:00, 1287.62it/s]


ContextualLinUCB_Tge0
Starting ContextualLinUCB_Tge0 Training
0.0 1.0
Ended ContextualLinUCB_Tge0 Training


ContextualLinUCB_Tge0: 100%|██████████| 18900/18900 [00:19<00:00, 992.42it/s] 


### Evaluating the models

In [12]:
# Cumulative Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [13]:
# Getting the results
cumulative_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    cumulative_results[agent_name] = hits_values


Evaluating LinearEGreedy

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.43 seconds executing Hits metric

Evaluating ContextualEgreedy_Te0

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.42 seconds executing Hits metric

Evaluating ContextualEgreedy_Tge0

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulat

In [14]:
df_cumulative = pd.DataFrame(columns=["Model", 5, 10, 20, 50, 100])
df_cumulative["Model"] = list(cumulative_results.keys())
df_cumulative.set_index("Model", inplace=True)
for agent_name, results in cumulative_results.items():
    df_cumulative.loc[agent_name] = [
        np.mean(list(metric_values.values())) for metric_values in results
    ]
df_cumulative

,5,10,20,50,100
Model,,,,,
LinearEGreedy,0.031746,0.126984,0.402116,2.857143,7.941799
ContextualEgreedy_Te0,0.534392,1.497354,3.724868,10.566138,19.391534
ContextualEgreedy_Tge0,0.687831,1.968254,4.592593,12.047619,20.857143
PTS,0.428571,0.714286,1.306878,4.312169,11.513228
ContextualPTS_Te0,0.534392,0.820106,1.507937,5.15873,12.460317
ContextualPTS_Tge0,0.529101,0.804233,1.52381,4.899471,11.962963
LinUCB,0.544974,1.047619,2.978836,10.910053,21.793651
ContextualLinUCB_Te0,1.42328,2.513228,6.798942,15.899471,25.566138
ContextualLinUCB_Tge0,1.42328,2.513228,6.798942,15.89418,25.582011
